# Create a whole brain mask 

by using the AROMAnonaggr runs of gamble control experiment

In [16]:
import os 
import pandas as pd
import glob
import numpy as np
import json
import nibabel as nib
import nilearn.plotting
import nilearn.masking as masking
import nilearn.image as image
from bids import BIDSLayout, BIDSValidator
from nipype.interfaces.base import Bunch

%matplotlib inline
import matplotlib.pyplot as plt

import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
from nipype.interfaces import utility as niu
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.modelgen as modelgen  # model generation


## Set Ups

In [2]:
# set paths for directories
user_path = os.path.expanduser('~')
home_path = os.path.join(user_path,'Desktop')

# control
project_path_control = os.path.join(home_path,'gamble_comb','GambleControl')
data_path_control = os.path.join(project_path_control,'data')
fmriprep_path_control = os.path.join(project_path_control,'data','derivatives')
output_path_control = os.path.join(project_path_control,'output');

# main experiment
project_path_gamble = os.path.join(home_path,'gamble_comb','Gamble')
data_path_gamble = os.path.join(project_path_gamble,'data')
fmriprep_path_gamble = os.path.join(project_path_gamble,'data','derivatives')
output_path_gamble = os.path.join(project_path_gamble,'output');

# note path
note_path = os.getcwd() # os.path.join(project_path_0,'runs','searchlight_lmer')

In [3]:
layout_control = BIDSLayout(data_path_control, derivatives = True)
layout_main = BIDSLayout(data_path_gamble, derivatives = True)

/opt/miniconda3/envs/fmri01/lib/python3.7/site-packages/bids/layout/validation.py:149: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [4]:
# subject to be removed
# gamble ['02','03','20']
drop_subjects = ['02','03','20']
gamble_subject = layout_main.get_subject()

for this_drop in drop_subjects:
    for this_subject in gamble_subject:
        if this_subject == this_drop:
            gamble_subject.remove(this_drop)

        
# specific runs to be dropped - which has not been implemented
drop_subjects = ['03','21','22']
control_subject = layout_control.get_subject()

for this_drop in drop_subjects:
    for this_subject in control_subject:
        if this_subject == this_drop:
            control_subject.remove(this_drop)

In [55]:
bold_file = layout_control.get(subject = control_subject,
                       extension = 'nii.gz', 
                       suffix = 'mask',
                       run = 1,
                       desc = 'brain', 
                       space = 'MNI152NLin6Asym', 
                       return_type = 'filename')
bold_file

['/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-01/func/sub-01_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-02/func/sub-02_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-04/func/sub-04_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-05/func/sub-05_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-06/func/sub-06_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-07/func/sub-07_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz',
 '/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives

In [56]:
mask_img = image.math_img('0*img',img = bold_file[0])

for this_bold in bold_file:
    print(this_bold)
    mask_img = image.math_img('img1 + img2',img1 = mask_img, img2 = this_bold)
   

/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-01/func/sub-01_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-02/func/sub-02_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-04/func/sub-04_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-05/func/sub-05_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-06/func/sub-06_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-07/func/sub-07_task-gamblecontrol_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
/Users/nova/Desktop/gamble_comb/GambleControl/data/derivatives/sub-08/func/sub-08_task-g

In [58]:
mask_img = image.math_img('img > 26.5',img = mask_img)
mask_img.to_filename(os.path.join('gray_matter_task-control_bold_mask.nii.gz'))

In [63]:
anat_file = layout_main.get(subject = gamble_subject,
                       extension = 'nii.gz', 
                       suffix = 'probseg', 
                       label = 'GM',
                       space = 'MNI152NLin6Asym', 
                       return_type = 'filename')

anat_mask = masking.compute_multi_brain_mask(anat_file, lower_bound = 0.5)
anat_mask = image.resample_to_img(anat_mask,'gray_matter_task-gamble_bold_mask.nii.gz')
nib.save(anat_mask,os.path.join('gray_matter_task-gamble_anat_mask.nii.gz'))

/opt/miniconda3/envs/fmri01/lib/python3.7/site-packages/nilearn/image/resampling.py:295: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  "Resampling binary images with continuous or "


In [64]:
anat_file = layout_control.get(subject = gamble_subject,
                       extension = 'nii.gz', 
                       suffix = 'probseg', 
                       label = 'GM',
                       space = 'MNI152NLin6Asym', 
                       return_type = 'filename')

anat_mask = masking.compute_multi_brain_mask(anat_file, lower_bound = 0.5)
anat_mask = image.resample_to_img(anat_mask,'gray_matter_task-control_bold_mask.nii.gz')
nib.save(anat_mask,os.path.join('gray_matter_task-control_anat_mask.nii.gz'))

/opt/miniconda3/envs/fmri01/lib/python3.7/site-packages/nilearn/image/resampling.py:295: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  "Resampling binary images with continuous or "


In [65]:
# conjuinction mask
for this_task in ['gamble','control']:
    
    anat_mask = nib.load('gray_matter_task-%s_anat_mask.nii.gz' %this_task)
    bold_mask = nib.load('gray_matter_task-%s_bold_mask.nii.gz' %this_task)

    mask_img = image.math_img('np.logical_and(img1,img2)',
                              img1 = anat_mask,
                              img2 = bold_mask)

    mask_img.to_filename('gray_matter_task-%s_anat_bold_mask.nii.gz' %this_task)